In [ ]:
import rasterio
from rasterio.windows import from_bounds
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def read_window(cog_path, bbox):
    with rasterio.open(cog_path) as src:
        window = from_bounds(*bbox, transform=src.transform)
        data = src.read(1, window=window, masked=True)
    return data

In [ ]:
def get_bbox(cog_path):
    with rasterio.open(cog_path) as src:
        bbox = src.bounds
    return bbox

In [ ]:
def create_small_bbox(original_bbox):
    # minLon, minLat, minLon + 0.008, minLat + 0.008]
    center_x = (original_bbox[0] + original_bbox[2]) / 2
    center_y = (original_bbox[1] + original_bbox[3]) / 2

    # Create a small bbox around the center
    small_bbox = [
        center_x - 0.004,
        center_y - 0.004,
        center_x + 0.004,
        center_y + 0.004,
    ]
    return small_bbox

In [ ]:
def return_small_bbox(cog_path):
    original_bbox = get_bbox(cog_path)
    small_bbox = create_small_bbox(original_bbox)
    return small_bbox

In [ ]:
def mean_value_calculation(cogs, bbox):
    arrays = []
    for cog in cogs:
        try:
            arr = read_window(cog, bbox)
            arrays.append(arr)
        except Exception as e:
            print(f"Error processing {cog}: {e}")
    if not arrays:
        print("No valid COGs processed.")
        return
    # Stack and calculate pixel-wise mean
    stack = np.ma.stack(arrays)
    mean_img = np.ma.mean(stack, axis=0)
    # Display
    plt.figure(figsize=(8, 6))
    plt.imshow(mean_img, cmap="viridis")
    plt.colorbar(label="Mean Value")
    plt.title("Pixel-wise Mean Across COGs")
    plt.xlabel("Column")
    plt.ylabel("Row")
    plt.show()

In [ ]:
bbox = return_small_bbox(cogs[0])

In [ ]:
mean_value_calculation(cogs, bbox)